In [1]:
import os

# 定义基础路径
base_dir = 'archive/Pest24/VOCdevkit/voc2007'
imagesets_dir = os.path.join(base_dir, 'ImageSets')
yolo_path_dir = os.path.join(base_dir, 'yolo_path')
images_dir = os.path.join(base_dir, 'images') # 假设图片在 images 目录

# 确保 yolo_path 目录存在
os.makedirs(yolo_path_dir, exist_ok=True)

# 处理 train, val, test 文件
for split in ['train', 'val', 'test']:
    imageset_file = os.path.join(imagesets_dir, f'{split}.txt')
    yolo_path_file = os.path.join(yolo_path_dir, f'{split}.txt')

    if not os.path.exists(imageset_file):
        print(f"警告：找不到文件 {imageset_file}")
        continue

    with open(imageset_file, 'r') as f:
        image_names = f.read().strip().split()

    # 写入相对路径
    with open(yolo_path_file, 'w') as f:
        for name in image_names:
            image_path = os.path.join(images_dir, f'{name}.jpg').replace('\\', '/')
            f.write(image_path + '\n')
    
    print(f"已成功生成 {yolo_path_file}")

print("\n所有路径文件已更新完毕。")


已成功生成 archive/Pest24/VOCdevkit/voc2007\yolo_path\train.txt
已成功生成 archive/Pest24/VOCdevkit/voc2007\yolo_path\val.txt
已成功生成 archive/Pest24/VOCdevkit/voc2007\yolo_path\test.txt

所有路径文件已更新完毕。


In [2]:
import os

# 返回到项目根目录
os.chdir('d:\\.FileModel')

# 列出 yolo_path 目录的内容
yolo_path_dir = 'archive/Pest24/VOCdevkit/voc2007/yolo_path'
if os.path.exists(yolo_path_dir):
    files = os.listdir(yolo_path_dir)
    print(f"'{yolo_path_dir}' 目录中的文件：")
    for file in files:
        print(file)
else:
    print(f"目录 '{yolo_path_dir}' 不存在。")

'archive/Pest24/VOCdevkit/voc2007/yolo_path' 目录中的文件：
test.cache
test.txt
train.cache
train.txt
val.txt


# 阶段一：Warmup 训练

本阶段只训练模型的头部，冻结主干网络，使用较小学习率，防止破坏预训练特征。

- 预训练模型：yolo11n.pt
- 数据集配置：pest24.yaml
- 冻结主干网络（backbone）
- 只训练头部（head）
- 学习率较小
- 训练周期较短
- 启用权重衰减、LoRA等正则化


In [2]:
import sys
import ultralytics
print(sys.executable)
print(ultralytics.__file__)
from ultralytics import YOLO
print("Successfully imported YOLO")

d:\.FileModel\.venv\Scripts\python.exe
d:\.FileModel\.venv\Lib\site-packages\ultralytics\__init__.py
Successfully imported YOLO


In [3]:
# 检查 GPU/CUDA 可用性并打印详细信息
import sys
import torch
import subprocess
import shutil

print(f"Python executable: {sys.executable}")
print(f"torch.__version__: {torch.__version__}")

cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")
print(f"CUDA device count: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    try:
        name = torch.cuda.get_device_name(i)
    except Exception:
        name = 'Unknown'
    print(f"GPU {i}: {name}")

# 尝试调用 nvidia-smi 获取更详细的状态（如果可用）
if shutil.which('nvidia-smi'):
    try:
        out = subprocess.check_output(['nvidia-smi', '--query-gpu=index,name,memory.total,memory.free,driver_version', '--format=csv,noheader,nounits'], stderr=subprocess.STDOUT)
        print('\nnvidia-smi output:')
        print(out.decode())
    except subprocess.CalledProcessError as e:
        print('nvidia-smi command failed:', e.output.decode() if e.output else e)
    except Exception as e:
        print('nvidia-smi query error:', e)
else:
    print('nvidia-smi not found in PATH')

# 建议用于 model.train 的 device 变量
device = 0 if cuda_available and torch.cuda.device_count() > 0 else 'cpu'
print(f"Recommended training device: {device}")

Python executable: d:\.FileModel\.venv\Scripts\python.exe
torch.__version__: 2.8.0+cu126
CUDA available: True
CUDA device count: 1
GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU

nvidia-smi output:
0, NVIDIA GeForce RTX 4060 Laptop GPU, 8188, 7527, 560.94

Recommended training device: 0


In [7]:
# 训练参数优化探索
import psutil
import torch
from math import ceil
import gc
import sys
import subprocess
from pathlib import Path
import datetime

# 主动清理内存
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

def estimate_batch_size():
    # 获取系统内存信息
    mem = psutil.virtual_memory()
    gpu_mem = 0
    if torch.cuda.is_available():
        gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3  # GB
    
    # 估算合理的batch size
    # 假设每张图片约占用 100MB 显存/内存
    if gpu_mem > 0:
        max_batch = int(gpu_mem * 0.7 / 0.1)  # 使用70%显存
    else:
        max_batch = int(mem.available / 1024**3 * 0.5 / 0.1)  # 使用50%可用内存
    
    # 确保batch size是8的倍数（有利于GPU优化）
    batch_size = max(8, min(32, 8 * ceil(max_batch/8)))
    return batch_size

def estimate_workers():
    # 获取CPU核心数
    cpu_count = psutil.cpu_count(logical=False)  # 物理核心数
    if cpu_count is None:
        cpu_count = psutil.cpu_count()  # 逻辑核心数
    
    # workers数量：通常设置为CPU核心数的1/2到1/4
    workers = max(1, min(8, cpu_count // 2))
    return workers

# 估算最佳参数
batch_size = estimate_batch_size()
num_workers = estimate_workers()

print(f"Estimated optimal batch size: {batch_size}")
print(f"Estimated optimal num_workers: {num_workers}")

# 显示当前系统资源状态
print("\nSystem Resources:")
print(f"CPU Usage: {psutil.cpu_percent()}%")
mem = psutil.virtual_memory()
print(f"Memory: {mem.percent}% used, {mem.available/1024**3:.1f}GB available")
if torch.cuda.is_available():
    print(f"GPU Memory: {torch.cuda.memory_allocated()/1024**3:.1f}GB allocated")

# 配置 TensorBoard
try:
    from torch.utils.tensorboard import SummaryWriter
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tensorboard"])  # 安装 tensorboard
    from torch.utils.tensorboard import SummaryWriter

# 全局变量来存储 writer 和日志目录
writer = None
TB_LOG_DIR = None

def setup_tensorboard(phase_name: str):
    """为训练阶段设置 TensorBoard。"""
    global writer, TB_LOG_DIR
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    TB_LOG_DIR = Path(f"runs/tensorboard/{phase_name}/{timestamp}")
    TB_LOG_DIR.mkdir(parents=True, exist_ok=True)
    writer = SummaryWriter(log_dir=str(TB_LOG_DIR))
    print(f"TensorBoard for '{phase_name}' phase initialized. Logs will be saved to: {TB_LOG_DIR}")


def _safe_scalar_log(prefix: str, data: dict, step: int):
    if not isinstance(data, dict) or writer is None:
        return
    for k, v in data.items():
        if isinstance(v, (int, float)):
            writer.add_scalar(f"{prefix}/{k}", v, step)


def tb_on_fit_epoch_end(trainer):
    """每个 epoch 结束时记录 metrics、loss、学习率与显存占用。"""
    if writer is None: return
    epoch = getattr(trainer, "epoch", None)
    if epoch is None:
        return

    # 训练/验证指标
    metrics = getattr(trainer, "metrics", None) or {}
    _safe_scalar_log("metrics", metrics, epoch)

    # 训练损失（有的版本将损失汇总到 metrics 中）
    losses = getattr(trainer, "loss", None)
    if isinstance(losses, dict):
        _safe_scalar_log("loss", losses, epoch)

    # 学习率
    try:
        if getattr(trainer, "optimizer", None):
            lr = trainer.optimizer.param_groups[0].get("lr", None)
            if isinstance(lr, (int, float)):
                writer.add_scalar("opt/lr", lr, epoch)
    except Exception:
        pass

    # GPU 显存（如可用）
    try:
        if torch.cuda.is_available():
            mem = torch.cuda.memory_allocated() / (1024 ** 2)  # MB
            writer.add_scalar("gpu/memory_allocated_mb", mem, epoch)
    except Exception:
        pass

    writer.flush()


def tb_on_train_end(trainer):
    """训练结束时的回调。"""
    global writer
    if writer:
        writer.close()
        print(f"TensorBoard logs saved to: {TB_LOG_DIR}")
        writer = None # 重置 writer
    
    # 训练结束后清理内存
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("TensorBoard callback functions defined and setup_tensorboard function is ready.")


Estimated optimal batch size: 32
Estimated optimal num_workers: 4

System Resources:
CPU Usage: 8.7%
Memory: 50.1% used, 7.6GB available
GPU Memory: 0.0GB allocated
TensorBoard callback functions defined and setup_tensorboard function is ready.


In [ ]:
# 加载模型并开始训练
from ultralytics import YOLO
import torch

# 1. 设置此阶段的 TensorBoard
setup_tensorboard("warmup")

# 2. 加载预训练模型
model = YOLO('yolo11n.pt')

# 3. 注册 TensorBoard 回调
try:
    model.add_callback("on_fit_epoch_end", tb_on_fit_epoch_end)
    model.add_callback("on_train_end", tb_on_train_end) # 使用 on_train_end
except NameError:
    print("TensorBoard callbacks not found. Please run the previous cell that defines them.")

# 4. 动态设备选择
_device = 0 if torch.cuda.is_available() else 'cpu'

# 5. 开始训练
results = model.train(
    data='pest24.yaml',
    epochs=10,                # warmup周期
    lr0=1e-4,                # 初始学习率
    dropout=0.2,             # Dropout
    weight_decay=0.01,       # 权重衰减
    batch=24,                 # 降低 batch_size
    workers=2,              # 降低 workers
    device=_device,          # 设备选择
    freeze=10,              # 冻结层数
    cache=True,            # 启用缓存
    overlap_mask=False,     # 减少mask计算开销
    profile=False,          # 关闭性能分析
    amp=True               # 自动混合精度训练
)

# 6. 保存warmup权重
model.save('yolo11n_warmup.pt')

print("\nWarmup complete. Launch TensorBoard with:\n  tensorboard --logdir runs/tensorboard\nThen open http://localhost:6006 in your browser.")


Ultralytics 8.3.208  Python-3.12.1 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=pest24.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=False, patience=3, perspective=0.0, plots=True, pose

In [ ]:
# 阶段二：Finetune 训练
from ultralytics import YOLO
import torch

# 1. 设置此阶段的 TensorBoard
setup_tensorboard("finetune")

# 2. 加载 warmup 阶段训练好的模型
model = YOLO('yolo11n_warmup.pt')

# 3. 注册 TensorBoard 回调
try:
    model.add_callback("on_fit_epoch_end", tb_on_fit_epoch_end)
    model.add_callback("on_train_end", tb_on_train_end) # 使用 on_train_end
except NameError:
    print("TensorBoard callbacks not found. Please run the cell that defines them.")

# 4. 动态设备选择
_device = 0 if torch.cuda.is_available() else 'cpu'

# 5. 开始 Finetune 训练
# 解冻所有层，使用更大的学习率
results = model.train(
    seed=42,
    data='pest24.yaml',
    epochs=100,               # 更多周期
    lr0=1e-3,                # 更大的学习率
    lrf=0.05,                 # 学习率衰减到 5%
    optimizer='AdamW',      # 使用 AdamW 优化器
    weight_decay=0.001,       # 权重衰减
    dropout=0.2,             # Dropout
    cos_lr=False,            # 余弦退火学习率
    patience=30,             # 更多耐心
    batch=24,        # 使用估算的最佳 batch_size
    workers=2,     # 使用估算的最佳 workers
    device=_device,
    freeze=0,                # 解冻所有层
    cache=True,
    amp=True,
    overlap_mask=False,      # 减少mask计算开销
    save_period=10,           # 周期保存模型
    name='finetune2'
)

# 6. 保存最终模型
model.save('yolo11n_final.pt')

print("\nFinetune complete. Launch TensorBoard with:\n  tensorboard --logdir runs/tensorboard\nThen open http://localhost:6006 in your browser.")


TensorBoard for 'finetune' phase initialized. Logs will be saved to: runs\tensorboard\finetune\20251012_160626
New https://pypi.org/project/ultralytics/8.3.211 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.208  Python-3.12.1 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=pest24.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.05, mask_ratio=4, max_det=300, mixup=0.0, mode=tr

单独的ECA注意力模块（已训练）

In [10]:
from ultralytics import YOLO
from eca_module import ECA
import torch.nn as nn

# === Step 1: 加载我们已有的最优模型 ===
model = YOLO("yolo11n_final4.pt")

# === Step 2: 插入 ECA 模块 ===
def inject_eca(model, layers=(4, 7, 10), k_size=3):
    """
    将 ECA 模块注入到 YOLO 模型的指定层之后。
    注意：此函数会直接修改传入的 model 对象。
    """
    # ultralytics 的模型层存储在 model.model.model (一个 nn.ModuleList)
    if not hasattr(model, 'model') or not hasattr(model.model, 'model'):
        print("⚠️ 无法找到模型层列表 (model.model.model)。模型结构可能已更改。")
        return model

    model_layers = model.model.model
    
    for i in layers:
        if i < 0:  # 支持负数索引
            i += len(model_layers)
            
        if not (0 <= i < len(model_layers)):
            print(f"⚠️ Layer index {i} out of range. Skipping.")
            continue

        old_layer = model_layers[i]
        
        # 尝试获取输出通道数
        ch = -1
        if isinstance(old_layer, nn.Sequential):
            # 查找最后一个卷积或瓶颈层来确定通道数
            for sub_layer in reversed(list(old_layer.children())):
                if hasattr(sub_layer, 'out_channels'):
                    ch = sub_layer.out_channels
                    break
                elif hasattr(sub_layer, 'cv2') and hasattr(sub_layer.cv2, 'conv'):
                    ch = sub_layer.cv2.conv.out_channels
                    break
        elif hasattr(old_layer, 'out_channels'):
            ch = old_layer.out_channels
        elif hasattr(old_layer, 'cv2') and hasattr(old_layer.cv2, 'conv'): # C2f
             ch = old_layer.cv2.conv.out_channels

        if ch == -1:
            print(f"⚠️ Could not determine output channels for layer {i}. Skipping.")
            continue
            
        # 直接在 ModuleList 中替换层
        model_layers[i] = nn.Sequential(old_layer, ECA(ch, k_size))
        print(f"✅ Inserted ECA after layer {i}, channels={ch}")
        
    return model

model = inject_eca(model, layers=[4, 7, 10])

# === Step 3: 保存注入后的结构版 ===
model.save("weights/yolov11n_eca_init.pt")

# === Step 3.5: 设置TensorBoard和回调 ===
# 直接在内存中的模型上设置，而不是重新加载
setup_tensorboard("module_injection")
try:
    model.add_callback("on_fit_epoch_end", tb_on_fit_epoch_end)
    model.add_callback("on_train_end", tb_on_train_end)
except NameError:
    print("TensorBoard callbacks not found. Please run the cell that defines them.")

_device = 0 if torch.cuda.is_available() else 'cpu'

# === Step 4: 在ECA版本上finetune ===
results = model.train(
    data="pest24.yaml",
    seed=42,
    epochs=50,
    batch=24,
    workers=2,
    lr0=1e-3,                
    lrf=0.05,                 
    optimizer="AdamW",
    weight_decay=0.001,
    dropout=0.2,
    cos_lr=True,
    cache=True,
    overlap_mask=False,
    amp=True,
    device=_device,
    patience=15,
    save_period=10,
    name='finetune_eca'
)

# === Step 5: 保存最终finetune权重 ===
model.save("weights/yolov11n_eca_finetuned.pt")


✅ Inserted ECA after layer 4, channels=128
⚠️ Could not determine output channels for layer 7. Skipping.
✅ Inserted ECA after layer 10, channels=256
TensorBoard for 'module_injection' phase initialized. Logs will be saved to: runs\tensorboard\module_injection\20251024_163421
New https://pypi.org/project/ultralytics/8.3.220 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.208  Python-3.12.1 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=pest24.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0

ECA单独使用（添加预热）

In [ ]:
from ultralytics import YOLO
from eca_module import ECA
import torch
import torch.nn as nn

# === Step 1: 加载我们已有的最优模型 ===
print("--- Loading baseline model: yolo11n_final4.pt ---")
model = YOLO("yolo11n_final4.pt")
print("Model loaded successfully.")

# === Step 2: 插入 ECA 模块 ===
def inject_eca(model, layers=(4, 7, 10), k_size=3):
    """
    将 ECA 模块注入到 YOLO 模型的指定层之后 (已修正对Conv层的支持)。
    """
    if not hasattr(model, 'model') or not hasattr(model.model, 'model'):
        print("⚠️ 无法找到模型层列表 (model.model.model)。")
        return model

    model_layers = model.model.model
    
    for i in layers:
        if i < 0: i += len(model_layers)
        if not (0 <= i < len(model_layers)):
            print(f"⚠️ Layer index {i} out of range. Skipping.")
            continue

        old_layer = model_layers[i]
        ch = -1
        
        # <<< MODIFICATION START: 增加对基础Conv模块的处理 >>>
        # 优先处理最具体的模块类型
        if hasattr(old_layer, 'conv') and isinstance(old_layer.conv, nn.Conv2d):
            # This handles the basic 'Conv' module from Ultralytics
            ch = old_layer.conv.out_channels
        # <<< MODIFICATION END >>>

        elif isinstance(old_layer, nn.Sequential):
            for sub_layer in reversed(list(old_layer.children())):
                if hasattr(sub_layer, 'out_channels'): ch = sub_layer.out_channels; break
                elif hasattr(sub_layer, 'cv2') and hasattr(sub_layer.cv2, 'conv'): ch = sub_layer.cv2.conv.out_channels; break
        elif hasattr(old_layer, 'cv2') and hasattr(old_layer.cv2, 'conv'): # C2f/C3
             ch = old_layer.cv2.conv.out_channels
        elif hasattr(old_layer, 'out_channels'): # Fallback for other potential layer types
            ch = old_layer.out_channels

        if ch == -1:
            print(f"⚠️ Could not determine output channels for layer {i} (Type: {type(old_layer).__name__}). Skipping.")
            continue
            
        model_layers[i] = nn.Sequential(old_layer, ECA(ch, k_size))
        print(f"✅ Inserted ECA after layer {i}, channels={ch}")
        
    return model

model = inject_eca(model, layers=[4, 7, 10])

# === Step 3: 设置TensorBoard和设备 ===
setup_tensorboard("eca_finetune_2stage1") # <<< MODIFICATION: 新的、清晰的实验命名
try:
    model.add_callback("on_fit_epoch_end", tb_on_fit_epoch_end)
    model.add_callback("on_train_end", tb_on_train_end)
except NameError:
    print("TensorBoard callbacks not found. Please run the cell that defines them.")
_device = 0 if torch.cuda.is_available() else 'cpu'


# <<< MODIFICATION START: 增加全新的第一阶段微调 >>>
# =======================================================
# === STAGE 1: Warmup New Modules (预热新模块) ===
# =======================================================
print("\n--- STAGE 1: WARMUP - Training ONLY the new ECA modules ---")

# 冻结所有层
print("Freezing all existing layers...")
for name, param in model.model.named_parameters():
    param.requires_grad = False

# 只解冻我们新加入的 ECA 模块的参数
print("Unfreezing only the ECA module parameters...")
unfrozen_count = 0
for m in model.model.modules():
    if isinstance(m, ECA):
        for param in m.parameters():
            param.requires_grad = True
        unfrozen_count += 1
print(f"Successfully unfrozen {unfrozen_count} ECA modules.")

# # 使用较小的学习率进行预热训练
# model.train(
#     data="pest24.yaml",
#     epochs=15,               # <<< MODIFICATION: 较短的预热周期
#     lr0=1e-4,                # <<< MODIFICATION: 较小的学习率
#     batch=24,
#     workers=2,
#     device=_device,
#     cache=True,
#     amp=True,
#     name='finetune_eca_stage1_warmup' # 清晰的阶段命名
# )
# print("--- STAGE 1: WARMUP Complete ---")

# <<< MODIFICATION: 保存第一阶段的权重 >>>
stage1_weights_path = "weights/yolov11n_eca_stage1_warmed_up.pt"
# model.save(stage1_weights_path)
print(f"Stage 1 weights saved to {stage1_weights_path}")
# <<< MODIFICATION END: 第一阶段微调结束 >>>

# =======================================================
# === STAGE 2: Global Fine-tuning (全局微调) ===
# =======================================================
print("\n--- STAGE 2: FINETUNE - Training all layers with a very low learning rate ---")

# <<< MODIFICATION: 从第一阶段的权重重新加载模型 >>>
print(f"--- Loading model from {stage1_weights_path} for Stage 2 ---")
model = YOLO(stage1_weights_path) # 重新加载

# <<< MODIFICATION: 为新模型对象重新注册回调 >>>
setup_tensorboard("eca_finetune_2stage2")
try:
    model.add_callback("on_fit_epoch_end", tb_on_fit_epoch_end)
    model.add_callback("on_train_end", tb_on_train_end)
except NameError:
    print("TensorBoard callbacks not found. Please run the cell that defines them.")

# 解冻所有层，以便进行全局微调
print("Unfreezing all layers for global fine-tuning...")
for name, param in model.model.named_parameters():
    param.requires_grad = True
print("All layers have been unfrozen.")

# 使用极小的学习率进行全局微调
results = model.train(
    data="pest24.yaml",
    seed=42,
    epochs=80,               # <<< MODIFICATION: 增加训练周期
    batch=24,
    workers=2,
    lr0=1e-4,                # <<< MODIFICATION: 显著提高初始学习率
    lrf=0.1,                 # <<< MODIFICATION: 调整最终学习率因子，避免下降过快
    optimizer="AdamW",
    weight_decay=0.001,
    dropout=0.2,
    cos_lr=False,
    cache=True,
    amp=True,
    device=_device,
    patience=20,             # <<< MODIFICATION: 相应增加耐心
    save_period=10,
    name='finetune_eca_stage2_global', # 清晰的阶段命名
    resume=True  # <<< MODIFICATION: 从上次中断处继续训练

)
print("--- STAGE 2: FINETUNE Complete ---")

# === Step 5: 保存最终Finetune权重 ===
final_weights_path = "weights/yolov11n_eca_2stage_finetuned.pt"
model.save(final_weights_path)
print(f"\nFinal model from two-stage fine-tuning saved to {final_weights_path}")



--- Loading baseline model: yolo11n_final4.pt ---
Model loaded successfully.
✅ Inserted ECA after layer 4, channels=128
✅ Inserted ECA after layer 7, channels=256
✅ Inserted ECA after layer 10, channels=256
TensorBoard for 'eca_finetune_2stage1' phase initialized. Logs will be saved to: runs\tensorboard\eca_finetune_2stage1\20251024_222146

--- STAGE 1: WARMUP - Training ONLY the new ECA modules ---
Freezing all existing layers...
Unfreezing only the ECA module parameters...
Successfully unfrozen 3 ECA modules.
Stage 1 weights saved to weights/yolov11n_eca_stage1_warmed_up.pt

--- STAGE 2: FINETUNE - Training all layers with a very low learning rate ---
--- Loading model from weights/yolov11n_eca_stage1_warmed_up.pt for Stage 2 ---
TensorBoard for 'eca_finetune_2stage2' phase initialized. Logs will be saved to: runs\tensorboard\eca_finetune_2stage2\20251024_222147
Unfreezing all layers for global fine-tuning...
All layers have been unfrozen.
New https://pypi.org/project/ultralytics/8.3

KeyboardInterrupt: 

单独的跨尺度融合(采样偏移)模块（需预热）（待训练）

In [ ]:
# train_with_csab.py
from ultralytics import YOLO
import torch.nn as nn
from ultralytics import YOLO
from csab_offset import CSAB_Offset

# === Step 1: 加载之前的最佳模型 ===
model = YOLO("yolo11n_final4.pt")

# === Step 2: 定义插入函数 (已修正递归错误) ===
def inject_csab_offset(model):
    """
    安全地将 CSAB_Offset 模块注入到指定的 Conv2d 层之后。
    采用“先收集，后修改”的策略以避免递归错误。
    """
    import torch.nn as nn
    from csab_offset import CSAB_Offset

    # 步骤 1: 收集需要修改的层的信息
    targets_to_modify = []
    for name, module in model.model.named_modules():
        # Ultralytics 的 Conv 模块通常是 'conv' 属性
        if hasattr(module, 'conv') and isinstance(module.conv, nn.Conv2d):
            conv_layer = module.conv
            if conv_layer.out_channels in [256, 512, 1024]:
                # 记录父模块和要替换的属性名 ('conv')
                targets_to_modify.append((module, 'conv', conv_layer))

    # 步骤 2: 对收集到的目标进行修改
    if not targets_to_modify:
        print("⚠️ 没有找到符合条件的 Conv2d 层进行注入。")
        return model

    print(f"找到了 {len(targets_to_modify)} 个符合注入条件的目标层。")
    for parent_module, attr_name, old_conv_layer in targets_to_modify:
        # 创建新的 CSAB_Offset 模块
        csab = CSAB_Offset(old_conv_layer.out_channels)
        # 创建一个新的序列，包含原始卷积层和我们的新模块
        new_sequential = nn.Sequential(old_conv_layer, csab)
        # 使用 setattr 安全地替换父模块中的原始卷积层
        setattr(parent_module, attr_name, new_sequential)
        print(f"✅ 成功在通道数为 {old_conv_layer.out_channels} 的层后注入 CSAB_Offset。")
        
    return model

# === Step 3: 注入模块 ===
model = inject_csab_offset(model)



# === Step 4: 设置TensorBoard和设备 ===
# (移除了保存初始权重文件的步骤)
setup_tensorboard("csab_finetune_2stage1") # <<< MODIFICATION
try:
    model.add_callback("on_fit_epoch_end", tb_on_fit_epoch_end)
    model.add_callback("on_train_end", tb_on_train_end)
except NameError:
    print("TensorBoard callbacks not found. Please run the cell that defines them.")

_device = 0 if torch.cuda.is_available() else 'cpu'

# <<< MODIFICATION START: 增加全新的第一阶段微调 >>>
# =======================================================
# === STAGE 1: Warmup New Modules (预热新模块) ===
# =======================================================
print("\n--- STAGE 1: WARMUP - Training ONLY the new CSAB_Offset modules ---")

# 冻结所有层
print("Freezing all existing layers...")
for name, param in model.model.named_parameters():
    param.requires_grad = False

# 只解冻我们新加入的 CSAB_Offset 模块的参数
print("Unfreezing only the CSAB_Offset module parameters...")
unfrozen_count = 0
for m in model.model.modules():
    if isinstance(m, CSAB_Offset):
        for param in m.parameters():
            param.requires_grad = True
        unfrozen_count += 1
print(f"Successfully unfrozen {unfrozen_count} CSAB_Offset modules.")

# 使用较小的学习率进行预热训练
model.train(
    data="pest24.yaml",
    epochs=15,               # 较短的预热周期
    lr0=1e-4,                # 较小的学习率
    batch=24,
    workers=2,
    device=_device,
    cache=True,
    amp=True,
    name='finetune_csab_stage1_warmup' # 清晰的阶段命名
)
print("--- STAGE 1: WARMUP Complete ---")
# <<< MODIFICATION: 保存第一阶段的权重 >>>
stage1_weights_path = "weights/yolov11n_csab_stage1_warmed_up.pt"
model.save(stage1_weights_path)
print(f"Stage 1 weights saved to {stage1_weights_path}")
# <<< MODIFICATION END: 第一阶段微调结束 >>>

# =======================================================
# === STAGE 2: Global Fine-tuning (全局微调) ===
# =======================================================
print("\n--- STAGE 2: FINETUNE - Training all layers with a very low learning rate ---")

# <<< MODIFICATION: 从第一阶段的权重重新加载模型 >>>
print(f"--- Loading model from {stage1_weights_path} for Stage 2 ---")
model = YOLO(stage1_weights_path) # 重新加载

# <<< MODIFICATION: 为新模型对象重新注册回调 >>>
setup_tensorboard("csab_finetune_2stage1") 
try:
    model.add_callback("on_fit_epoch_end", tb_on_fit_epoch_end)
    model.add_callback("on_train_end", tb_on_train_end)
except NameError:
    print("TensorBoard callbacks not found. Please run the cell that defines them.")

# 解冻所有层
print("Unfreezing all layers for global fine-tuning...")
for name, param in model.model.named_parameters():
    param.requires_grad = True
print("All layers have been unfrozen.")

# 使用极小的学习率进行全局微调
model.train(
    data="pest24.yaml",
    seed=42,
    epochs=50,
    batch=24,
    workers=2,
    lr0=1e-4,                # <<< MODIFICATION: 关键改动！使用极小的学习率
    lrf=0.1,                # <<< MODIFICATION: 最终学习率也相应降低
    optimizer="AdamW",
    weight_decay=0.001,
    dropout=0.2,
    cos_lr=True,
    cache=True,
    amp=True,
    device=_device,
    patience=15,
    save_period=10,
    name='finetune_csab_stage2_global' # 清晰的阶段命名
)
print("--- STAGE 2: FINETUNE Complete ---")

# === Step 6: 保存最终finetune完成的模型 ===
final_weights_path = "weights/yolov11n_csab_2stage_finetuned.pt" # <<< MODIFICATION
model.save(final_weights_path)
print(f"\nFinal model from two-stage fine-tuning saved to {final_weights_path}")

单独使用DBL_TSD模块（待训练）

In [ ]:
# train_with_dbl_corrected.py
from ultralytics import YOLO
from ultralytics.models.yolo.detect.train import DetectionTrainer # <<< MODIFICATION: 导入基础训练器
from dbl_loss import DBL_TSD # 确保您的 dbl_loss_tsd.py 在项目目录中
import torch

# === Step 1: 定义一个使用DBL损失的自定义训练器 ===
class DBLDetectionTrainer(DetectionTrainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.custom_loss = DBL_TSD(cls_w=1.0, box_w=5.0, iou_w=2.0, scale_alpha=1.0, trunc_thresh=0.4)
        print("✅ DBLDetectionTrainer initialized with custom DBL_TSD loss function.")

    def get_loss(self, batch, preds):
        loss, loss_items = self.criterion(preds, batch)
        if hasattr(self.criterion, 'assigned_targets'):
             targets = self.criterion.assigned_targets
        else:
            return loss, loss_items
        custom_loss_total, custom_loss_info = self.custom_loss(preds, targets)
        final_loss_items = torch.cat((
            custom_loss_info['box_loss'].unsqueeze(0),
            custom_loss_info['cls_loss'].unsqueeze(0),
            loss_items[2:]
        ))
        return custom_loss_total, final_loss_items

# === Step 2: 加载上一次训练的best模型 ===
print("--- Loading baseline model: yolo11n_final4.pt ---")
model = YOLO("yolo11n_final4.pt")
print("Model loaded successfully.")


# === Step 3: 设置TensorBoard和设备 ===
setup_tensorboard("dbl_finetune_global")
try:
    model.add_callback("on_fit_epoch_end", tb_on_fit_epoch_end)
    model.add_callback("on_train_end", tb_on_train_end)
except NameError:
    print("TensorBoard callbacks not found.")
_device = 0 if torch.cuda.is_available() else 'cpu'


# === MODIFICATION: 采用全局适应性微调策略 ===
print("\n--- Starting Global Adaptive Fine-tuning with DBL_TSD Loss ---")
print("All layers will be trained from the start to adapt to the new loss function.")

model.train(
    trainer=DBLDetectionTrainer,   # 指定使用我们自定义的训练器
    data="pest24.yaml",
    seed=42,
    epochs=50,                     # 完整的训练周期
    batch=24,
    workers=2,
    lr0=5e-5,                      # <<< MODIFICATION: 使用一个适中的初始学习率
    lrf=1e-6,                      # 最终学习率可以保持较低
    optimizer="AdamW",
    weight_decay=0.001,
    patience=15,
    name='finetune_dbl_global_adapt', # 新的、清晰的实验命名
    device=_device,
    cache=True,
    amp=True
    # 注意：这里没有 freeze 参数，所有层默认都是可训练的
)

# === Step 4: 保存最终模型 ===
final_weights_path = "weights/yolo11n_dbl_global_finetuned.pt"
model.save(final_weights_path)
print(f"\nFinal model from global adaptive fine-tuning saved to {final_weights_path}")